# Week 11 Assignment

# PART 1 - Pivoting Data for Fun!

In this exercise, we're going to work with data about condemned buildings in St. Louis.  This data set is in `/data/condemn.txt` in CSV format.

## STEP 1: Load and Filter data

For this analysis, we want to look only at a relevent range of years and only years that are in teh past.  So, let's first take a look at the data between 1986 and 2014 (inclusive of those dates).

At the end of this step, you should have a data frame loaded from the condemnations file and filtered down to only the data between 1986 and 2014.  Put that into a dataframe named `condemnations`

**Some helpful notes:**
* Use the columns `InspectDate` to filter this date range

In [ ]:
%matplotlib inline
import pandas as pd

condemnations = pd.read_csv('/data/condemn.txt')
condemnations.shape

In [ ]:
condemnations.columns[[2,8,35,36]]

In [ ]:
condemnations = pd.read_csv('/data/condemn.txt', converters={
    'AddrAdjusted': str,
    'AddrNum': str,
    'Inspector': str,
    'Images': str
})
condemnations.columns

In [ ]:
condemnations['InspectDate'].head()

In [ ]:
date_filter = (condemnations['InspectDate'].str[0:4] >= '1986') & (condemnations['InspectDate'].str[0:4] <= '2014')

In [ ]:
date_filter.value_counts()

In [ ]:
condemnations = condemnations[date_filter]

In [ ]:
assert(condemnations.shape == (30232, 40))
assert(len(condemnations[condemnations['InspectDate'] < '1986'])==0)
assert(len(condemnations[condemnations['InspectDate'] >= '2015'])==0)

## STEP 2: Pivot and Summarize

In this step we want to pivot our data so that we have one row per year, a column for each value that appears in the `Status` column, and a count of how many condemnations occurred of each type in each year.  Put your summary pivot table into a variable called `by_year`.

**Some helpful notes:**
* For some reason, Pandas is just kind of stubborn about count being the aggregate function.  You may need to tell it to `np.ma.count` specifically.  I've imported Numpy as np for you.

In [ ]:
import numpy as np

condemnations['Year'] = condemnations['InspectDate'].str[0:4]
by_year = pd.pivot_table(condemnations,
    values='AcctNum',
    columns='Status',
    index='Year',
    aggfunc='count')

In [ ]:
by_year

In [ ]:
assert(by_year.shape == (29, 4))
assert(list(by_year.columns) == ['B', 'L', 'R', 'W'])
assert(list(by_year.sum()) == [10572, 4966, 98, 9670])

In [ ]:
by_year

## STEP 3: Plot and see what you see

Use Pandas built-in plotting capabilities to visualize this information in way that you think makes sense.  It might be that plotting a simple multi-series line chart would give you some insights.  Show your plot in the cell below, then use the text cell below that to write a few sentences about what you see.


In [ ]:
%matplotlib inline
by_year.plot()

---

# PART 2 - Video Conference Usage

In this next part of the assignment, we'll be looking at how video conference usage has change for my company over the past couple of months.  This real data.  Please don't judge anyone based on the information here.  ;)

For the assignment, I want you to compare (by person) the number of meetings hosted by each user for the last two weeks of February and the last two weeks of March.  This might be a little complicated to think through HOW to do that, so the steps below are one approach.

1. Create a new column called startDate that contains just the day part of the startTime column in the data.  You can do that using datetime functions if you want, but you'll notice that the date in this file is a string formatted YYYY-MM-DD-HH:mm:ss, so it's probably easier just to pull out the first 10 characters of the string instead.  `meetings['startTime'].str[0:10]`
2. Create another new column.  This one should contain a label like "BEFORE" for the week 2/17 - 2/28 and "DURING" for the week of 3/16 - 3/27.  A hand tip on this is that you can set values for some rows in a data frame using `meetings.loc[row_filter,'Column Name'] = 'Value'` if you've created a `row_filter` as we discussed in prior weeks.
3. From here, using `pivot_table()` is one good way to summarize the information.

**FINAL RESULT**

For a final result, I'd like you to produce a data frame named **summary** that contains:
* One ROW for each userName.  The userName can either be the row index or a column in the data frame.
* One COLUMN for the "BEFORE" week and one COLUMN for the "DURING" week.  Name those columns 'Before' and 'During'.
* One more calculated column that shows the percent increase from BEFORE to DURING.  Name that column 'pctChange'.


In [1]:
import pandas as pd
import numpy as np
meetings = pd.read_csv('/data/meetings.csv')
meetings
meetings.dtypes
meetings.columns





Index(['meeting_uuid', 'meetingTitle', 'meetingId', 'userName', 'user_id',
       'user_tags', 'endTime', 'startTime', 'email', 'participantSeconds',
       'joinDate', 'joinWeek', 'joinMonth', 'participants',
       'participantMinutes', 'meetingDurationMinutes', 'popId', 'userType',
       'billable', 'moderatorLess', 'start_time', 'total_highlights_created',
       'smart_meeting', 'transcription_used',
       'total_transcription_duration_minutes'],
      dtype='object')

In [2]:
meetings=meetings[['userName', 'startTime']]
meetings

,userName,startTime
0,chris.rogers.amitechsolutions,2020-03-31 21:00:22
1,Drew.Marco,2020-03-31 20:58:41
2,eric.keeney,2020-03-31 20:57:51
3,amit.bhagat,2020-03-31 20:34:43
4,andrea.bryant.amitechsolutions,2020-03-31 20:30:02
...,...,...
1183,eric.keeney,2020-01-02 15:28:08
1184,jeff.hatfield.amitechsolutions,2020-01-02 15:01:32
1185,jennifer.patton.amitechs,2020-01-02 14:56:32
1186,jeff.hatfield.amitechsolutions,2020-01-02 14:22:28


In [3]:
from datetime import datetime, timedelta

In [4]:
meetings['startDate']= meetings['startTime'].str[0:10]
 


#meetings['endTime', "startTime", "joinDate", "joinWeek", "start_time" ]= pd.to_datetime(meetings["endTime", "startTime", "joinDate", "joinWeek", "start_time"], format="%Y/%m/%d, %H:%M:%S")
#meetings['endTime' ]= pd.to_datetime(meetings["endTime"], format="%Y/%m/%d, %H:%M:%S")

In [5]:
meetings['startDate']=pd.to_datetime(meetings['startDate'])
meetings

,userName,startTime,startDate
0,chris.rogers.amitechsolutions,2020-03-31 21:00:22,2020-03-31
1,Drew.Marco,2020-03-31 20:58:41,2020-03-31
2,eric.keeney,2020-03-31 20:57:51,2020-03-31
3,amit.bhagat,2020-03-31 20:34:43,2020-03-31
4,andrea.bryant.amitechsolutions,2020-03-31 20:30:02,2020-03-31
...,...,...,...
1183,eric.keeney,2020-01-02 15:28:08,2020-01-02
1184,jeff.hatfield.amitechsolutions,2020-01-02 15:01:32,2020-01-02
1185,jennifer.patton.amitechs,2020-01-02 14:56:32,2020-01-02
1186,jeff.hatfield.amitechsolutions,2020-01-02 14:22:28,2020-01-02


In [6]:
before= meetings[meetings['startDate'].isin(pd.date_range('2020-02-17', '2020-02-28'))]
during= meetings[meetings['startDate'].isin(pd.date_range('2020-03-16', '2020-03-27'))]
during

,userName,startTime,startDate
59,angie.warren,2020-03-27 22:00:23,2020-03-27
60,Drew.Marco,2020-03-27 20:58:29,2020-03-27
61,paul.boal,2020-03-27 20:29:51,2020-03-27
62,shawn.reilly.amitechsolutions,2020-03-27 19:56:12,2020-03-27
63,amit.bhagat,2020-03-27 19:53:30,2020-03-27
...,...,...,...
332,tejus.maduskar,2020-03-16 14:13:29,2020-03-16
333,michael.demos,2020-03-16 14:06:32,2020-03-16
334,tejus.maduskar,2020-03-16 13:45:15,2020-03-16
335,raghavender.tuniki,2020-03-16 13:31:15,2020-03-16


In [7]:
totalB= before.groupby(['userName']).size()
#totalB=pd.DataFrame(totalB)
totalB
#totalB=totalB.rename(columns={0:'NumBefore'})

#totalD=during.groupby(['userName']).size()
#totalD.totalD.to_frame().reset_index()



userName
Drew.Marco                        12
amit.bhagat                       17
andrea.bryant.amitechsolutions     5
angie.warren                       5
chris.rogers.amitechsolutions      4
eric.keeney                       15
eric.rodman                        4
jeff.hatfield.amitechsolutions    13
jennifer.patton.amitechs           5
jessica.sikora                     3
matt.allsup                        5
matt.brown.amitechsolutions        1
melissa.menne                     10
michael.demos                      5
paul.boal                         15
paul.smith.amitechsolutions       23
pete.cornwell                      1
raghavender.tuniki                 6
sandip.shaw                        9
santhosh.kanala                    2
shawn.reilly.amitechsolutions      7
tejus.maduskar                    12
dtype: int64

In [8]:
totalD=during.groupby(['userName']).size()
#totalD.totalD.to_frame().reset_index()
#totalD=pd.DataFrame(totalD)
#totalD.rename(columns ={'0':'NumDuring'})
totalD

userName
Drew.Marco                        19
amit.bhagat                       30
andrea.bryant.amitechsolutions     3
angie.warren                       9
chris.rogers.amitechsolutions      3
eric.keeney                       27
eric.rodman                       12
jeff.hatfield.amitechsolutions    26
jennifer.patton.amitechs           7
jessica.sikora                     7
matt.brown.amitechsolutions        2
melissa.menne                      9
michael.demos                     11
paul.boal                         35
paul.smith.amitechsolutions       10
pete.cornwell                      9
raghavender.tuniki                 2
sandip.shaw                        2
santhosh.kanala                    6
shawn.reilly.amitechsolutions     10
tejus.maduskar                    39
dtype: int64

In [23]:

pctChange=(totalD-totalB)/totalB
pctChange
pctChange.round(2)
pctChange=pctChange.round(2)
pctChange

,0
userName,
Drew.Marco,0.58
amit.bhagat,0.76
andrea.bryant.amitechsolutions,-0.40
angie.warren,0.80
chris.rogers.amitechsolutions,-0.25
eric.keeney,0.80
eric.rodman,2.00
jeff.hatfield.amitechsolutions,1.00
jennifer.patton.amitechs,0.40


In [24]:
totalD=pd.DataFrame(totalD)
totalB=pd.DataFrame(totalB)
pctChange=pd.DataFrame(pctChange)

In [25]:
summary=pd.merge(totalB, totalD, how='outer', on='userName')
summary=summary.merge(pctChange, how='outer', on='userName')
#summary.columns['Before', 'During','pctChange']
summary.rename(columns={'0_x':'Before', '0_y':'During', '0':'pctChange'}, inplace=True)
summary.rename(index={2:'pctChange'})
summary.columns=['Before', 'During', 'pctChange']
summary=summary.fillna(0)
summary.columns

Index(['Before', 'During', 'pctChange'], dtype='object')

In [26]:
summary['pctChange'].mean().round(2)

0.96

---

In [ ]:
assert(len(summary) == 22)

In [ ]:
assert(summary['pctChange'].mean().round(2) == 0.92)

---

## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Save this note with Ctrl-S (or Cmd-S)
2. Skip down to the last command cell (the one starting with `%%bash`) and run that cell.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [ ]:
assert False, "DO NOT REMOVE THIS LINE"

---

In [27]:
%%bash
git pull
git add week11_assignment_2.ipynb
git commit -a -m "Submitting the week 10 programming assignment"
git push

Already up to date.
[main fd96e19] Submitting the week 10 programming assignment
 2 files changed, 1160 insertions(+), 6 deletions(-)
 create mode 100644 week11/week11_assignment_2.ipynb


To github.com:skuca/hds5210-2021.git
   ba50eb4..fd96e19  main -> main



---

If the message above says something like _Submitting the week 8 programming assignment_ or _Everything is up to date_, then your work was submitted correctly.